#Estrutura dos Arquivos

In [ ]:
#-l (list) do curl lista o conteúdo da pasta e não baixa um arquivo específico.

!curl -l ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/

DC
EE
EF
EP
EQ
GM
HB
IN
LT
PF
RC
SR
ST


| **Código** | **Descrição** |
|------------|--------------|
| **DC** | Dados Complementares |
| **EE** | Equipamentos |
| **EF** | Equipes |
| **EP** | Estabelecimentos de Saúde |
| **EQ** | Equipes de Saúde Bucal |
| **GM** | Gestão Municipal |
| **HB** | Habilitações |
| **IN** | Incentivos |
| **LT** | Leitos |
| **PF** | Profissionais de Saúde |
| **RC** | Recurso Complementar |
| **SR** | Serviços Especializados |
| **ST** | Serviços Especializados Tipo |

#CNES - Profissional

###Aquisição dos Links

In [ ]:
import json

# Definição do caminho base dos arquivos no FTP
link_base = "ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/PF/PFMG"
link_suffix = ".dbc"
anos = range(2019, 2021) # Define os anos desejados
meses = range(1, 2) # Define os meses desejados

# Lista para armazenar os links de download
links = []

# Gerando os links corretamente
for ano in anos:
    for mes in meses:
        ano_sufixo = str(ano)[2:]
        mes_sufixo = str(mes).zfill(2)
        links.append(f"{link_base}{ano_sufixo}{mes_sufixo}{link_suffix}")

# Salvar corretamente como lista
with open("links.json", "w") as f:
    json.dump(links, f, indent=4)

In [ ]:
links

['ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/PF/PFMG1901.dbc',
 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/PF/PFMG2001.dbc']

###Aquisição do Arquivo DBC

In [ ]:
import json
import os

# Ler para garantir que foi salvo corretamente
with open("links.json", "r") as f:
    links = json.load(f)

In [ ]:
# Criar a pasta de armazenamento se não existir
os.makedirs("bases_raw", exist_ok=True)

In [ ]:
# Baixar os arquivos do FTP para a pasta "bases_raw"
for link in links:
    os.system(f"wget -P bases_raw --inet4-only {link}")

###Conversão em CSV

In [ ]:
# Criar a pasta de saída dos arquivos convertidos
os.makedirs("bases_descomprimidas", exist_ok=True)

In [ ]:
%reload_ext rpy2.ipython
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'

In [ ]:
%%R

# O jeito normal `install.packages("read.dbc")` não funciona pois a biblioteca foi removida do CRAN
# Essa forma instala o código direto do github
devtools::install_github("danicat/read.dbc")

In [ ]:
%%R
library("read.dbc")

# Função para converter todos os arquivos .dbc na pasta
convert_all_dbc <- function(rawDir, convertedDir) {
    files <- list.files(path = rawDir, pattern = "\\.dbc$", full.names = TRUE)  # Lista arquivos .dbc
    for (file in files) {
        tryCatch({
            print(paste("Convertendo:", file))  # Exibir arquivo atual

            # Ler arquivo .dbc
            x <- read.dbc(file)

            # Criar nome do arquivo CSV
            csv_file <- paste0(convertedDir, "/", tools::file_path_sans_ext(basename(file)), ".csv")

            # Salvar CSV garantindo delimitadores corretos
            write.table(x, file = csv_file, fileEncoding = "UTF-8", row.names = FALSE, sep = ",", dec = ".", na = "", quote = TRUE)

            print(paste("Convertido com sucesso:", csv_file))
        }, error = function(e) {
            print(paste("Erro ao converter:", file, "->", e$message))
        })
    }
}

# Chamar a função
convert_all_dbc("bases_raw", "bases_descomprimidas")

###Compilação todos arquivos

In [ ]:
import pandas as pd

In [ ]:
# Criar a pasta de saída se não existir
os.makedirs("bases_compiladas", exist_ok=True)

In [ ]:
# Listar todos os arquivos CSV na pasta bases_descomprimidas
csv_files = [f for f in os.listdir("bases_descomprimidas") if f.endswith(".csv")]

In [ ]:
# Criar um dataframe vazio para armazenar os dados
df_compilado = pd.DataFrame()

In [ ]:
# Percorrer todos os arquivos CSV e fazer o append no dataframe compilado
for file in csv_files:
    file_path = os.path.join("bases_descomprimidas", file)
    print(f"Lendo {file_path}")
    df = pd.read_csv(file_path, dtype=str)  # Lendo os dados como string para evitar problemas de tipo
    df_compilado = pd.concat([df_compilado, df], ignore_index=True)

In [ ]:
# Salvar o arquivo compilado na nova pasta
output_file = "bases_compiladas/dados_compilados.csv"
df_compilado.to_csv(output_file, index=False, encoding="utf-8")

#CNES - Equipe

###Aquisição dos Links

In [ ]:
import json

# Definição do caminho base dos arquivos no FTP
link_base = "ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/EF/EFMG"
link_suffix = ".dbc"
anos = range(2019, 2021) # Define os anos desejados
meses = range(1, 2) # Define os meses desejados

# Lista para armazenar os links de download
links = []

# Gerando os links corretamente
for ano in anos:
    for mes in meses:
        ano_sufixo = str(ano)[2:]
        mes_sufixo = str(mes).zfill(2)
        links.append(f"{link_base}{ano_sufixo}{mes_sufixo}{link_suffix}")

# Salvar corretamente como lista
with open("links.json", "w") as f:
    json.dump(links, f, indent=4)

In [ ]:
links

['ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/EF/EFMG1901.dbc',
 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/EF/EFMG2001.dbc']

###Aquisição do Arquivo DBC

In [ ]:
import json
import os

# Ler para garantir que foi salvo corretamente
with open("links.json", "r") as f:
    links = json.load(f)

In [ ]:
# Criar a pasta de armazenamento se não existir
os.makedirs("bases_raw", exist_ok=True)

In [ ]:
# Baixar os arquivos do FTP para a pasta "bases_raw"
for link in links:
    os.system(f"wget -P bases_raw --inet4-only {link}")

###Conversão em CSV

In [ ]:
# Criar a pasta de saída dos arquivos convertidos
os.makedirs("bases_descomprimidas", exist_ok=True)

In [ ]:
%reload_ext rpy2.ipython
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'

In [ ]:
%%R

# O jeito normal `install.packages("read.dbc")` não funciona pois a biblioteca foi removida do CRAN
# Essa forma instala o código direto do github
devtools::install_github("danicat/read.dbc")

── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpTstRmy/remotes61968824da/danicat-read.dbc-90970b3/DESCRIPTION’ ... OK
* preparing ‘read.dbc’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘read.dbc_1.0.7.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


In [ ]:
%%R
library("read.dbc")

# Função para converter todos os arquivos .dbc na pasta
convert_all_dbc <- function(rawDir, convertedDir) {
    files <- list.files(path = rawDir, pattern = "\\.dbc$", full.names = TRUE)  # Lista arquivos .dbc
    for (file in files) {
        tryCatch({
            print(paste("Convertendo:", file))  # Exibir arquivo atual

            # Ler arquivo .dbc
            x <- read.dbc(file)

            # Criar nome do arquivo CSV
            csv_file <- paste0(convertedDir, "/", tools::file_path_sans_ext(basename(file)), ".csv")

            # Salvar CSV garantindo delimitadores corretos
            write.table(x, file = csv_file, fileEncoding = "UTF-8", row.names = FALSE, sep = ",", dec = ".", na = "", quote = TRUE)

            print(paste("Convertido com sucesso:", csv_file))
        }, error = function(e) {
            print(paste("Erro ao converter:", file, "->", e$message))
        })
    }
}

# Chamar a função
convert_all_dbc("bases_raw", "bases_descomprimidas")

[1] "Convertendo: bases_raw/EFMG1901.dbc"
[1] "Convertido com sucesso: bases_descomprimidas/EFMG1901.csv"
[1] "Convertendo: bases_raw/EFMG2001.dbc"
[1] "Convertido com sucesso: bases_descomprimidas/EFMG2001.csv"


###Compilação todos arquivos

In [ ]:
import pandas as pd

In [ ]:
# Criar a pasta de saída se não existir
os.makedirs("bases_compiladas", exist_ok=True)

In [ ]:
# Listar todos os arquivos CSV na pasta bases_descomprimidas
csv_files = [f for f in os.listdir("bases_descomprimidas") if f.endswith(".csv")]

In [ ]:
# Criar um dataframe vazio para armazenar os dados
df_compilado = pd.DataFrame()

In [ ]:
# Percorrer todos os arquivos CSV e fazer o append no dataframe compilado
for file in csv_files:
    file_path = os.path.join("bases_descomprimidas", file)
    print(f"Lendo {file_path}")
    df = pd.read_csv(file_path, dtype=str)  # Lendo os dados como string para evitar problemas de tipo
    df_compilado = pd.concat([df_compilado, df], ignore_index=True)

Lendo bases_descomprimidas/EFMG2001.csv
Lendo bases_descomprimidas/EFMG1901.csv


In [ ]:
# Salvar o arquivo compilado na nova pasta
output_file = "bases_compiladas/dados_compilados.csv"
df_compilado.to_csv(output_file, index=False, encoding="utf-8")